In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
toxic = pd.read_csv('dirty.csv')
clean = pd.read_csv('clean.csv')

toxic['tag'] = 1
clean['tag'] = 0
toxic = toxic.drop(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)
clean = clean.drop(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)

In [9]:
data = pd.concat([clean, toxic], ignore_index=True).sample(frac=1, random_state=1).reset_index(drop=True)


In [10]:
print(data)

                                             comment_text  tag
0       Yes I read the explanations. And what you say ...    0
1       Image:Aerotow_launch_MGC.jpg listed for deleti...    0
2       okay its fine. the image wasn't downloaded at ...    0
3       Just now writing here but I like Austin said, ...    0
4       Deletion review for Tyler Clementi==\nAn edito...    0
...                                                   ...  ...
154246  GA reassessment of Multiple sequence alignment...    0
154247  2014 (UTC)\n\nToday, many people from countrie...    0
154248  What's that 12.205.247.27 deletion? \n\nI see ...    0
154249  Yes, I must second this.  Scottsdale is not at...    0
154250  How was the external link inappropriate? Why d...    0

[154251 rows x 2 columns]


In [12]:
train = data[0:int(len(data)*0.95)]
test = data[int(len(data)*0.95):-1]

In [11]:
import re, string
#re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): 
    re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return re_tok.sub(r' \1 ', s).split()

In [15]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train['comment_text'])
test_term_doc = vec.transform(test['comment_text'])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [17]:
x = trn_term_doc
test_x = test_term_doc

In [20]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=False)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [21]:
label_cols = ['tag']

preds = np.zeros((len(test), len(label_cols)))
array = []
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    array.append([m,r])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit tag


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
import pickle

with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(m, file)
with open('r_values.pkl', 'wb') as file:
    pickle.dump(r, file)


In [26]:
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vec, file)

In [41]:
class Judge():
    def __init__(self):
        with open('judge_model/tfidf_vectorizer.pkl', 'rb') as file:
            vec = pickle.load(file)
        with open('judge_model/logistic_regression_model.pkl', 'rb') as file:
            model = pickle.load(file)
        with open('judge_model/r_values.pkl', 'rb') as file:
            r = pickle.load(file)
            
    def judge(self, words):
        tokens = vec.transform([words])
        preds = m.predict_proba(tokens.multiply(r))[:,1]
        
        return 1 if preds>0.5 else 0

In [42]:
test = Judge()

In [44]:
print(test.judge("fuck you"))

1
